In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

../scripts/functions.py:121: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [3]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

1


In [4]:
hex_.lattice_coor

array([[ 1.999995, -2.886748],
       [ 2.999995, -2.886748],
       [ 3.999995, -2.886748],
       [ 4.999995, -2.886748],
       [ 2.500005, -3.175429],
       [ 3.500005, -3.175429],
       [ 4.500005, -3.175429],
       [ 5.500005, -3.175429],
       [ 0.499995, -0.288672],
       [ 1.499995, -0.288672],
       [ 2.499995, -0.288672],
       [ 3.499995, -0.288672],
       [ 1.000005, -0.577353],
       [ 2.000005, -0.577353],
       [ 3.000005, -0.577353],
       [ 4.000005, -0.577353],
       [ 0.999995, -1.154698],
       [ 1.999995, -1.154698],
       [ 2.999995, -1.154698],
       [ 3.999995, -1.154698],
       [ 1.500005, -1.443379],
       [ 2.500005, -1.443379],
       [ 3.500005, -1.443379],
       [ 4.500005, -1.443379],
       [ 1.499995, -2.020723],
       [ 2.499995, -2.020723],
       [ 3.499995, -2.020723],
       [ 4.499995, -2.020723],
       [ 2.000005, -2.309404],
       [ 3.000005, -2.309404],
       [ 4.000005, -2.309404],
       [ 5.000005, -2.309404]], dtype=f

### set params to 0 (represent ground state for v=1)

In [5]:
ma._ws[:] = 0
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

### create Markov Chain 
number of markov chain is denoted by n_samples_

In [6]:
n_jobs = 12

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
_ = sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.

number of core : 12


In [7]:
n_samples_ = 10000
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [8]:
import scripts.dynamics as Dynamics
d = Dynamics.new_dynamics(op, ma)
%time P_ = d.multiprocess(samples_state, 300, 12) 

CPU times: user 57.7 ms, sys: 100 ms, total: 158 ms
Wall time: 3.01 s


In [9]:
P = P_[0]
T = P_[1]

# t_list = np.linspace(0,50,100)
t_list = np.arange(0,20,0.1)
P_list = f.process_P(P, T, t_list)

In [10]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))

In [11]:
from importlib import reload
reload(f)


operators = f.return_vison(hi, np.arange(hex_.lattice_coor.shape[0]))

../scripts/functions.py:121: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [13]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1 = operators[0].get_conn_flattened(P_list[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real

size = len(operators)

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


for s in range(size):
    print(s)
    
    if operators[s]:
        sub2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)[1].mean().real
        _, mels2_ = operators[s].get_conn_flattened(P_list_, sections2)
        mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[s] = (mels2 * mels1).mean(axis=1)
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [14]:
from plotly import graph_objects as go

In [27]:


data = go.Scatter(
    x = t_list,
    y = dimer_corr[4]
)
Fig = go.Figure(data = [data])

In [28]:
Fig.show()

In [17]:
# import struct
# with open('your_data.dat', 'wb') as fe:  
#     for i in range(dimer_corr.shape[0]):
#         fe.write(struct.pack('{}\n'.format(dimer_corr[i])))
        

In [18]:
outfile = open('table1.dat', 'w')
 # write table header
for i in range(dimer_corr.shape[0]):
    outfile.write('#{}\n'.format(i)) 
    for n in range(dimer_corr.shape[1]):
        outfile.write('{}\t{:.2f}\t{}\n'.format(n, t_list[n], dimer_corr[i,n]))
    outfile.write('\n') 

In [19]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((81,N),dtype=np.complex128)
n = -1
import re
with open('K4_4V1.0st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr1.append(float(match[2]))
                corr2[n,i] = float(match[2]) + 1j*float(match[3])
                i+=1
            except:
                pass

In [20]:
dimer_corr[1]

array([0.32873149, 0.32793117, 0.32893157, 0.32573029, 0.31972789,
       0.31852741, 0.31752701, 0.30952381, 0.30492197, 0.29771909,
       0.29231693, 0.28871549, 0.28531413, 0.27971188, 0.27691076,
       0.28071228, 0.27290916, 0.26790716, 0.26510604, 0.26270508,
       0.2615046 , 0.26010404, 0.25110044, 0.2444978 , 0.24009604,
       0.24009604, 0.23429372, 0.23289316, 0.22809124, 0.22669068,
       0.22088836, 0.21888756, 0.21348539, 0.20248099, 0.20068027,
       0.19527811, 0.18487395, 0.18587435, 0.18107243, 0.17446979,
       0.17286915, 0.16846739, 0.16866747, 0.16446579, 0.16146459,
       0.16546619, 0.16526611, 0.16866747, 0.16246499, 0.16046419,
       0.15646259, 0.15486194, 0.15206082, 0.14705882, 0.14245698,
       0.14005602, 0.13885554, 0.1372549 , 0.12945178, 0.1322529 ,
       0.12845138, 0.12164866, 0.12364946, 0.12184874, 0.12264906,
       0.12144858, 0.12064826, 0.11884754, 0.1132453 , 0.11364546,
       0.11304522, 0.11244498, 0.11004402, 0.1082433 , 0.10464